This notebook uses sklearn to train a logistic regression classifier on the IMDB sentiment analysis dataset available from the [huggingface datasets library](https://huggingface.co/datasets/imdb) and described in [Maas et al. 2011](https://aclanthology.org/P11-1015.pdf)

## Import Relevant Packages

In [4]:
#pip install sklearn
#pip install datasets

In [1]:
import sklearn
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

C:\Users\laram\AppData\Local\Continuum\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
C:\Users\laram\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


## Importing and Preparing Datasets

In [2]:
raw_datasets = load_dataset("imdb")
# print(raw_datasets['train'][0])
# print(raw_datasets['test'][0])

Reusing dataset imdb (C:\Users\laram\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [42]:
len(raw_datasets['train'])

25000

Let's convert the training data into the format expected by scikit-learn - a list of input vectors (documents) and a list of associated output labels.

In [3]:
# Converting the training data from nested dictionary to two separate lists

train_dataset = raw_datasets['train']
train_data = []
train_data_labels = []
for item in train_dataset:
    train_data.append(item['text'])
    train_data_labels.append(item['label'])

We'll use the CountVectorizer class to extract the words as features. Each document is represented as a 200 dimension vector of word counts. Only the 200 most frequent words are used in this version.

In [4]:
vectorizer = CountVectorizer(analyzer='word',max_features=200,lowercase=True)
features = vectorizer.fit_transform(train_data)
features_nd = features.toarray()

In [5]:
# Split the training data into a training and validation (dev) set.
X_train, X_val, y_train, y_val = train_test_split(features_nd,train_data_labels,train_size=0.8,random_state=123)

## Training The Logistic Regression Model

In [7]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 4, 0, 0]], dtype=int64)

In [10]:
sum(y_train)

9963

In [6]:
log_model = LogisticRegression()
log_model = log_model.fit(X=X_train,y=y_train)

C:\Users\laram\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Test the model on the validation set.

In [35]:
# Testing the training data on the model
y_pred = log_model.predict(X_val)
print(accuracy_score(y_val,y_pred))

0.7666


C:\Users\laram\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


Now let's prepare some test data. Use the same 1000 as in the BERT notebook.

In [38]:
test_dataset = raw_datasets['test'].shuffle(seed=42).select(range(1000)) # randomly selecting 1000 / 25000
test_data = []
test_data_labels = []
for item in test_dataset:
    test_data.append(item['text'])
    test_data_labels.append(item['label'])

Apply the model to the test data.

In [39]:
test_pred=log_model.predict(vectorizer.transform(test_data).toarray())

C:\Users\laram\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


In [40]:
print(accuracy_score(test_pred,test_data_labels))

0.74
